In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Thesis

Mounted at /content/drive
/content/drive/MyDrive/Thesis


In [2]:
archaic = []
modern = []
with open("dataset/archaic.src", "r", encoding="utf-8") as f:
    for line in f:
        archaic.append(line.strip())
with open("dataset/modern.tgt", "r", encoding="utf-8") as f:
    for line in f:
        modern.append(line.strip())

length = len(archaic)

In [3]:
all_pos = []
all_morph = []
with open("all_pos.txt", "r", encoding="utf-8") as f:
    for line in f:
      line = line.strip().replace("[", "").replace("]", "").replace("'", "").replace(" ", "")
      pos_of_sent = []
      for pos in line.split(","):
        pos_of_sent.append(pos)
      all_pos.append(pos_of_sent)

with open("all_morph.txt", "r", encoding="utf-8") as f:
    for line in f:
      line = line.strip().replace("[", "").replace("]", "").replace("'", "").replace(" ", "")
      morph_of_sent = []
      for morph in line.split(","):
        morph_of_sent.append(morph)
      all_morph.append(morph_of_sent)

print(all_pos[0])

['PROPN', 'NOUN', 'VERB', 'CCONJ', 'VERB', 'VERB', 'VERB', 'PRON', 'ADV', 'NOUN', 'NOUN', 'VERB', 'VERB', 'NOUN', 'NOUN', 'SCONJ', 'NOUN', 'PART', 'VERB', 'PROPN', 'VERB', 'VERB', 'NOUN', 'PUNCT', 'ADV', 'VERB', 'NOUN', 'NOUN', 'PUNCT', 'ADV', 'VERB', 'VERB', 'CCONJ', 'PROPN', 'VERB', 'PUNCT']


# Create POS and MORPH embeddings

In [4]:
from collections import defaultdict

pos_lookup = defaultdict(lambda: len(pos_lookup))
morph_lookup = defaultdict(lambda: len(morph_lookup))

pos_embeddings = []
morph_embeddings = []

for pos_seq, morph_seq in zip(all_pos, all_morph):
    pos_ids = [pos_lookup[pos] for pos in pos_seq]
    pos_ids = [101] + pos_ids + [102]
    morph_ids = [morph_lookup[morph] for morph in morph_seq]
    morph_ids = [101] + morph_ids + [102]
    pos_embeddings.append(pos_ids)
    morph_embeddings.append(morph_ids)

In [5]:
pos_vocab_size = len(pos_lookup) + 2
morph_vocab_size = len(morph_lookup) + 2

In [6]:
print(pos_embeddings[-1])

[101, 2, 2, 3, 2, 2, 1, 8, 2, 2, 1, 8, 1, 5, 2, 4, 1, 6, 1, 2, 0, 8, 2, 1, 2, 12, 1, 8, 102]


# New Bart Model


In [7]:
import torch
import torch.nn as nn
from transformers import AutoConfig

In [8]:
from transformers import AutoTokenizer, BertTokenizer, BartForConditionalGeneration, Text2TextGenerationPipeline, BartConfig
# tokenizer = BertTokenizer.from_pretrained("fnlp/bart-base-chinese")
tokenizer = BertTokenizer.from_pretrained("RUCAIBox/Erya")
# tokenizer = AutoTokenizer.from_pretrained("Jihuai/bert-ancient-chinese")
config = BartConfig.from_pretrained("fnlp/bart-base-chinese")
# text2text_generator = Text2TextGenerationPipeline(model, tokenizer)
# text2text_generator("北京是[MASK]的首都", max_length=50, do_sample=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/259k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

In [9]:
from transformers.models.bart.modeling_bart import *

In [10]:
from transformers.modeling_attn_mask_utils import (
    _prepare_4d_attention_mask,
    _prepare_4d_attention_mask_for_sdpa,
    _prepare_4d_causal_attention_mask,
    _prepare_4d_causal_attention_mask_for_sdpa,
)

In [11]:
# embed_dim = config.d_model
# t = nn.Embedding(256, embed_dim, 0)
# # e = nn.Embedding(config.vocab_size, embed_dim, 0)
# print(t)
# print(train_pos_ids.shape)
# print(train_pos_ids[0].shape)
# print(t(train_pos_ids[0]))
# # print(t(torch.tensor([0,1,2])))

In [12]:
import random
class CustomBartEncoder(BartEncoder):
    def __init__(self, config: BartConfig, embed_tokens: Optional[nn.Embedding] = None):
        super().__init__(config)
        self.dropout = config.dropout
        self.layerdrop = config.encoder_layerdrop

        embed_dim = config.d_model
        self.padding_idx = config.pad_token_id
        self.max_source_positions = config.max_position_embeddings
        self.embed_scale = math.sqrt(embed_dim) if config.scale_embedding else 1.0

        self.embed_tokens = nn.Embedding(config.vocab_size, embed_dim, self.padding_idx)

        if embed_tokens is not None:
            self.embed_tokens.weight = embed_tokens.weight

        self.embed_positions = BartLearnedPositionalEmbedding(
            config.max_position_embeddings,
            embed_dim,
        )
        self.layers = nn.ModuleList([BartEncoderLayer(config) for _ in range(config.encoder_layers)])
        self._use_flash_attention_2 = config._attn_implementation == "flash_attention_2"
        self._use_sdpa = config._attn_implementation == "sdpa"
        self.layernorm_embedding = nn.LayerNorm(embed_dim)

        self.gradient_checkpointing = False
        # Initialize weights and apply final processing
        self.post_init()
        self.pos_embedding = nn.Embedding(256,embed_dim,0)  # 添加的新embeddings
        self.morph_embedding = nn.Embedding(256,embed_dim,0)

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        pos_ids = None, # 新的輸入特徵
        morph_ids = None, # 新的輸入特徵
    ) -> Union[Tuple, BaseModelOutput]:

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # retrieve input_ids and inputs_embeds
        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input = input_ids
            input_ids = input_ids.view(-1, input_ids.shape[-1])
        elif inputs_embeds is not None:
            input = inputs_embeds[:, :, -1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        if inputs_embeds is None:
            inputs_embeds = self.embed_tokens(input_ids) * self.embed_scale

        embed_pos = self.embed_positions(input)
        embed_pos = embed_pos.to(inputs_embeds.device)

        pos_embedding = self.pos_embedding(pos_ids)
        morph_embedding = self.morph_embedding(morph_ids)

        hidden_states = inputs_embeds + embed_pos + pos_embedding + morph_embedding # 全部加在一起
        hidden_states = self.layernorm_embedding(hidden_states)
        hidden_states = nn.functional.dropout(hidden_states, p=self.dropout, training=self.training)

        # expand attention_mask
        if attention_mask is not None:
            if self._use_flash_attention_2:
                attention_mask = attention_mask if 0 in attention_mask else None
            elif self._use_sdpa and head_mask is None and not output_attentions:
                # output_attentions=True & head_mask can not be supported when using SDPA, fall back to
                # the manual implementation that requires a 4D causal mask in all cases.
                # [bsz, seq_len] -> [bsz, 1, tgt_seq_len, src_seq_len]
                attention_mask = _prepare_4d_attention_mask_for_sdpa(attention_mask, inputs_embeds.dtype)
            else:
                # [bsz, seq_len] -> [bsz, 1, tgt_seq_len, src_seq_len]
                attention_mask = _prepare_4d_attention_mask(attention_mask, inputs_embeds.dtype)

        encoder_states = () if output_hidden_states else None
        all_attentions = () if output_attentions else None

        # check if head_mask has a correct number of layers specified if desired
        if head_mask is not None:
            if head_mask.size()[0] != (len(self.layers)):
                raise ValueError(
                    f"The head_mask should be specified for {len(self.layers)} layers, but it is for"
                    f" {head_mask.size()[0]}."
                )

        for idx, encoder_layer in enumerate(self.layers):
            if output_hidden_states:
                encoder_states = encoder_states + (hidden_states,)
            # add LayerDrop (see https://arxiv.org/abs/1909.11556 for description)
            to_drop = False
            if self.training:
                dropout_probability = torch.rand([])
                if dropout_probability < self.layerdrop:  # skip the layer
                    to_drop = True

            if to_drop:
                layer_outputs = (None, None)
            else:
                if self.gradient_checkpointing and self.training:
                    layer_outputs = self._gradient_checkpointing_func(
                        encoder_layer.__call__,
                        hidden_states,
                        attention_mask,
                        (head_mask[idx] if head_mask is not None else None),
                        output_attentions,
                    )
                else:
                    layer_outputs = encoder_layer(
                        hidden_states,
                        attention_mask,
                        layer_head_mask=(head_mask[idx] if head_mask is not None else None),
                        output_attentions=output_attentions,
                    )

                hidden_states = layer_outputs[0]

            if output_attentions:
                all_attentions = all_attentions + (layer_outputs[1],)

        if output_hidden_states:
            encoder_states = encoder_states + (hidden_states,)

        if not return_dict:
            return tuple(v for v in [hidden_states, encoder_states, all_attentions] if v is not None)
        return BaseModelOutput(
            last_hidden_state=hidden_states, hidden_states=encoder_states, attentions=all_attentions
        )

In [13]:
from transformers import BartModel

In [14]:
class BartWithPOSMorph(BartModel):
    def __init__(self, config: BartConfig):
        super().__init__(config)
        padding_idx, vocab_size = config.pad_token_id, config.vocab_size
        self.shared = nn.Embedding(vocab_size, config.d_model, padding_idx)
        self.encoder = CustomBartEncoder(config, self.shared)
        self.decoder = BartDecoder(config, self.shared)

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        decoder_input_ids=None,
        decoder_attention_mask=None,
        head_mask=None,
        decoder_head_mask=None,
        cross_attn_head_mask=None,
        encoder_outputs=None,
        past_key_values=None,
        inputs_embeds=None,
        decoder_inputs_embeds=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        pos_ids=None,  #自加的
        morph_ids=None,
    )-> Union[Tuple, Seq2SeqModelOutput]:
        # different to other models, Bart automatically creates decoder_input_ids from
        # input_ids if no decoder_input_ids are provided
        if decoder_input_ids is None and decoder_inputs_embeds is None:
            if input_ids is None:
                raise ValueError(
                    "If no `decoder_input_ids` or `decoder_inputs_embeds` are "
                    "passed, `input_ids` cannot be `None`. Please pass either "
                    "`input_ids` or `decoder_input_ids` or `decoder_inputs_embeds`."
                )

            decoder_input_ids = shift_tokens_right(
                input_ids, self.config.pad_token_id, self.config.decoder_start_token_id
            )

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if encoder_outputs is None:
          encoder_outputs = self.encoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                head_mask=head_mask,
                inputs_embeds=inputs_embeds,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
                pos_ids=pos_ids, # 自加的，傳給 Encoder
                morph_ids=morph_ids, # 傳給 Encoder
            )
        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            encoder_outputs = BaseModelOutput(
                last_hidden_state=encoder_outputs[0],
                hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
                attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None,
            )

        # decoder outputs consists of (dec_features, past_key_value, dec_hidden, dec_attn)
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            attention_mask=decoder_attention_mask,
            encoder_hidden_states=encoder_outputs[0],
            encoder_attention_mask=attention_mask,
            head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            past_key_values=past_key_values,
            inputs_embeds=decoder_inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        if not return_dict:
            return decoder_outputs + encoder_outputs

        return Seq2SeqModelOutput(
            last_hidden_state=decoder_outputs.last_hidden_state,
            past_key_values=decoder_outputs.past_key_values,
            decoder_hidden_states=decoder_outputs.hidden_states,
            decoder_attentions=decoder_outputs.attentions,
            cross_attentions=decoder_outputs.cross_attentions,
            encoder_last_hidden_state=encoder_outputs.last_hidden_state,
            encoder_hidden_states=encoder_outputs.hidden_states,
            encoder_attentions=encoder_outputs.attentions,
        )

In [15]:
class CustomBartForConditionalGeneration(BartForConditionalGeneration):
    base_model_prefix = "model"
    _tied_weights_keys = ["encoder.embed_tokens.weight", "decoder.embed_tokens.weight", "lm_head.weight"]
    _keys_to_ignore_on_load_missing = ["final_logits_bias"]

    def __init__(self, config: BartConfig):
        super().__init__(config)
        self.model = BartWithPOSMorph(config) # 替換成修改過後的類別
        self.register_buffer("final_logits_bias", torch.zeros((1, self.model.shared.num_embeddings)))
        self.lm_head = nn.Linear(config.d_model, self.model.shared.num_embeddings, bias=False)

        # Initialize weights and apply final processing
        self.post_init()

    # 修改 prepare_inputs_for_generation，提供新的特徵輸入
    # prepare_inputs_for_generation 定義了 Model.generate 如何提供資料
    def prepare_inputs_for_generation(
        self,
        decoder_input_ids,
        past_key_values=None,
        attention_mask=None,
        decoder_attention_mask=None,
        head_mask=None,
        decoder_head_mask=None,
        cross_attn_head_mask=None,
        use_cache=None,
        encoder_outputs=None,
        pos_ids=None,  # 提供輸入特徵給模型
        morph_ids=None,
        **kwargs
    ):
        # cut decoder_input_ids if past_key_values is used
        if past_key_values is not None:
            past_length = past_key_values[0][0].shape[2]

            # Some generation methods already pass only the last input ID
            if decoder_input_ids.shape[1] > past_length:
                remove_prefix_length = past_length
            else:
                # Default to old behavior: keep only final ID
                remove_prefix_length = decoder_input_ids.shape[1] - 1

            decoder_input_ids = decoder_input_ids[:, remove_prefix_length:]

        return {
            "input_ids": None,  # encoder_outputs is defined. input_ids not needed
            "encoder_outputs": encoder_outputs,
            "past_key_values": past_key_values,
            "decoder_input_ids": decoder_input_ids,
            "attention_mask": attention_mask,
            "decoder_attention_mask": decoder_attention_mask,
            "head_mask": head_mask,
            "decoder_head_mask": decoder_head_mask,
            "cross_attn_head_mask": cross_attn_head_mask,
            "use_cache": use_cache,  # change this to avoid caching (presumably for debugging)
            "pos_ids": pos_ids,  # 自加的
            "morph_ids": morph_ids
        }

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        decoder_head_mask: Optional[torch.Tensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[List[torch.FloatTensor]] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        pos_ids = None, # 提供輸入特徵給模型
        morph_ids = None
    ) -> Union[Tuple, Seq2SeqLMOutput]:

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if labels is not None:
            if use_cache:
                logger.warning("The `use_cache` argument is changed to `False` since `labels` is provided.")
            use_cache = False
            if decoder_input_ids is None and decoder_inputs_embeds is None:
                decoder_input_ids = shift_tokens_right(
                    labels, self.config.pad_token_id, self.config.decoder_start_token_id
                )

        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            encoder_outputs=encoder_outputs,
            decoder_attention_mask=decoder_attention_mask,
            head_mask=head_mask,
            decoder_head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            decoder_inputs_embeds=decoder_inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            pos_ids=pos_ids, # 自加的 提供輸入特徵給模型
            morph_ids=morph_ids
        )
        lm_logits = self.lm_head(outputs[0])
        lm_logits = lm_logits + self.final_logits_bias.to(lm_logits.device)

        masked_lm_loss = None
        if labels is not None:
            labels = labels.to(lm_logits.device)
            loss_fct = CrossEntropyLoss()
            masked_lm_loss = loss_fct(lm_logits.view(-1, self.config.vocab_size), labels.view(-1))

        if not return_dict:
            output = (lm_logits,) + outputs[1:]
            return ((masked_lm_loss,) + output) if masked_lm_loss is not None else output

        return Seq2SeqLMOutput(
            loss=masked_lm_loss,
            logits=lm_logits,
            past_key_values=outputs.past_key_values,
            decoder_hidden_states=outputs.decoder_hidden_states,
            decoder_attentions=outputs.decoder_attentions,
            cross_attentions=outputs.cross_attentions,
            encoder_last_hidden_state=outputs.encoder_last_hidden_state,
            encoder_hidden_states=outputs.encoder_hidden_states,
            encoder_attentions=outputs.encoder_attentions,
        )

In [16]:
model = CustomBartForConditionalGeneration.from_pretrained('fnlp/bart-base-chinese') # 加載預訓練權重

model.safetensors:   0%|          | 0.00/561M [00:00<?, ?B/s]

Some weights of CustomBartForConditionalGeneration were not initialized from the model checkpoint at fnlp/bart-base-chinese and are newly initialized: ['model.encoder.morph_embedding.weight', 'model.encoder.pos_embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load the data

In [17]:
train_x = archaic[:7000]
train_y = modern[:7000]
train_pos_ids = pos_embeddings[:7000]
train_morph_ids = morph_embeddings[:7000]

dev_x = archaic[7000:9000]
dev_y = modern[7000:9000]
dev_pos_ids = pos_embeddings[7000:9000]
dev_morph_ids = morph_embeddings[7000:9000]

test_x = archaic[9000:]
test_y = modern[9000:]
test_pos_ids = pos_embeddings[9000:]
test_morph_ids = morph_embeddings[9000:]
print(len(train_x))
print(len(dev_x))
print(len(test_x))

7000
2000
2045


In [18]:
train_input_ids = tokenizer(train_x, return_tensors='pt', padding=True, max_length=256, truncation=True)['input_ids']

train_labels = tokenizer(train_y, return_tensors='pt', padding=True, max_length=256, truncation=True)['input_ids']

train_attention_mask = tokenizer(train_x, return_tensors='pt', padding=True, max_length=256, truncation=True)['attention_mask']

In [19]:
from torch.nn.utils.rnn import pad_sequence

train_pos_ids = [torch.tensor(pos_ids) for pos_ids in train_pos_ids]
train_pos_ids = pad_sequence(train_pos_ids, batch_first=True, padding_value=0)
train_pos_ids = train_pos_ids[:, :256]

train_morph_ids = [torch.tensor(morph_ids) for morph_ids in train_morph_ids]
train_morph_ids = pad_sequence(train_morph_ids, batch_first=True, padding_value=0)
train_morph_ids = train_morph_ids[:, :256]

In [20]:
print(train_input_ids.shape)
print(train_labels.shape)
print(train_attention_mask.shape)
print(train_pos_ids.shape)


torch.Size([7000, 256])
torch.Size([7000, 256])
torch.Size([7000, 256])
torch.Size([7000, 256])


In [21]:
dev_input_ids = tokenizer(dev_x, return_tensors='pt', padding=True, max_length=256, truncation=True)['input_ids']

dev_labels = tokenizer(dev_y, return_tensors='pt', padding=True, max_length=256, truncation=True)['input_ids']

dev_attention_mask = tokenizer(dev_x, return_tensors='pt', padding=True, max_length=256, truncation=True)['attention_mask']

dev_pos_ids = [torch.tensor(pos_ids) for pos_ids in dev_pos_ids]
dev_pos_ids = pad_sequence(dev_pos_ids, batch_first=True, padding_value=0)
dev_pos_ids = dev_pos_ids[:, :256]

dev_morph_ids = [torch.tensor(morph_ids) for morph_ids in dev_morph_ids]
dev_morph_ids = pad_sequence(dev_morph_ids, batch_first=True, padding_value=0)
dev_morph_ids = dev_morph_ids[:, :256]

print(dev_input_ids.shape)
print(dev_labels.shape)
print(dev_attention_mask.shape)
print(dev_pos_ids.shape)

torch.Size([2000, 119])
torch.Size([2000, 217])
torch.Size([2000, 119])
torch.Size([2000, 119])


In [22]:
test_input_ids = tokenizer(test_x, return_tensors='pt', padding=True, max_length=156, truncation=True)['input_ids']

test_labels = tokenizer(test_y, return_tensors='pt', padding=True, max_length=256, truncation=True)['input_ids']

test_attention_mask = tokenizer(test_x, return_tensors='pt', padding=True, max_length=156, truncation=True)['attention_mask']

test_pos_ids = [torch.tensor(pos_ids) for pos_ids in test_pos_ids]
test_pos_ids = pad_sequence(test_pos_ids, batch_first=True, padding_value=0)
test_pos_ids = test_pos_ids[:, :156]

test_morph_ids = [torch.tensor(morph_ids) for morph_ids in test_morph_ids]
test_morph_ids = pad_sequence(test_morph_ids, batch_first=True, padding_value=0)
test_morph_ids = test_morph_ids[:, :156]

print(test_input_ids.shape)
print(test_labels.shape)
print(test_attention_mask.shape)
print(test_pos_ids.shape)

torch.Size([2045, 156])
torch.Size([2045, 256])
torch.Size([2045, 156])
torch.Size([2045, 156])


In [23]:
from torch.utils.data import Dataset, DataLoader
class TranslationDataset(Dataset):
    def __init__(self, archaic_ids, modern_ids, pos_tags, morph_tags): #, attention_masks):
        self.archaic_ids = archaic_ids
        self.modern_ids = modern_ids
        self.pos_tags = pos_tags
        self.morph_tags = morph_tags
        # self.attention_masks = attention_masks

    def __len__(self):
        return len(self.archaic_ids)

    def __getitem__(self, index):
        archaic_id = self.archaic_ids[index]
        modern_id = self.modern_ids[index]
        pos_tag = self.pos_tags[index]
        morph_tag = self.morph_tags[index]
        # attention_mask = self.attention_masks[index]
        return archaic_id, modern_id, pos_tag, morph_tag #, attention_mask

train_dataset = TranslationDataset(train_input_ids, train_labels, train_pos_ids, train_morph_ids)#, train_attention_mask)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)

dev_dataset = TranslationDataset(dev_input_ids, dev_labels, dev_pos_ids, dev_morph_ids)#, dev_attention_mask)
dev_dataloader = DataLoader(dev_dataset, batch_size=16, shuffle=False)

test_dataset = TranslationDataset(test_input_ids, test_labels, test_pos_ids, test_morph_ids)#, test_attention_mask)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Training

In [24]:
from transformers import AdamW
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 10

device = torch.device("cuda")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
!pip install wandb
from tqdm import tqdm
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00


In [26]:
# # prompt: calculate_bleu

# from nltk.translate.bleu_score import sentence_bleu

# def calculate_bleu(generated_sentences, reference_sentences):
#     """
#     Calculates the BLEU score for the given generated and reference sentences.

#     Args:
#         generated_sentences: A list of generated sentences.
#         reference_sentences: A list of reference sentences.

#     Returns:
#         The BLEU score.
#     """

#     bleu_scores = []
#     for generated_sentence, reference_sentence in zip(generated_sentences, reference_sentences):
#         reference_tokens = reference_sentence.split()
#         generated_tokens = generated_sentence.split()
#         bleu_scores.append(sentence_bleu([reference_tokens], generated_tokens))

#     return sum(bleu_scores) / len(bleu_scores)


In [27]:
model.to(device)
wandb.init(project='bart-pos-morph', config=config)

# 配置超参数
config = wandb.config
config.num_epochs = 10
config.batch_size = 16
config.learning_rate = 1e-5

# 训练模型
for epoch in range(config.num_epochs):
    model.train()
    train_loss = 0
    train_steps = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1} Training")
    for batch in progress_bar:
        input_ids, output_ids, pos_ids, morph_ids = batch
        input_ids = input_ids.to(device)
        output_ids = output_ids.to(device)
        pos_ids = pos_ids.to(device)
        morph_ids = morph_ids.to(device)

        outputs = model(input_ids=input_ids, labels=output_ids, pos_ids=pos_ids, morph_ids=morph_ids)
        logits = outputs.logits
        # loss = criterion(logits.view(-1, logits.size(-1)), output_ids[:, 1:].contiguous().view(-1))
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()
        train_steps += 1
        progress_bar.set_postfix({'loss': loss.item()})

    train_loss /= train_steps
    wandb.log({'epoch': epoch + 1, 'train_loss': train_loss})

    # 在验证集上评估模型
    model.eval()
    eval_loss = 0
    eval_steps = 0
    progress_bar = tqdm(dev_dataloader, desc=f"Epoch {epoch + 1} Evaluation")

    with torch.no_grad():
        for batch in progress_bar:
            input_ids, output_ids, pos_ids, morph_ids = batch
            input_ids = input_ids.to(device)
            output_ids = output_ids.to(device)
            pos_ids = pos_ids.to(device)
            morph_ids = morph_ids.to(device)

            outputs = model(input_ids=input_ids, labels=output_ids, pos_ids=pos_ids, morph_ids=morph_ids)
            logits = outputs.logits
            # loss = criterion(logits.view(-1, logits.size(-1)), output_ids[:, 1:].contiguous().view(-1))
            loss = outputs.loss

            eval_loss += loss.item()
            eval_steps += 1
            progress_bar.set_postfix({'loss': loss.item()})

            # pred = model.generate(input_ids=input_ids, pos_ids=pos_ids, morph_ids=morph_ids, do_sample=False, num_beams=5)
            # generated_sentences.extend(tokenizer.batch_decode(pred, skip_special_tokens=True))

    eval_loss /= eval_steps
    wandb.log({'epoch': epoch + 1, 'eval_loss': eval_loss})

    # 评估生成的翻译结果
    # bleu_score = calculate_bleu(generated_sentences, dev_y)
    # wandb.log({'epoch': epoch + 1, 'bleu_score': bleu_score})

wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch 10 Evaluation: 100%|██████████| 125/125 [00:10<00:00, 12.42it/s, loss=0.534]


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
eval_loss,█▅▃▃▂▂▁▁▁▁
train_loss,█▃▃▃▂▂▂▁▁▁
epoch,10
eval_loss,0.37382
train_loss,0.29655


In [28]:
!pip install evaluate
!pip install bert_score
import evaluate
bertscore = evaluate.load("bertscore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl 

In [29]:
predicted_translations = []
with torch.no_grad():
  for batch in tqdm(test_dataloader, desc="Testing"):
    input_ids, _, pos_ids, morph_ids = batch
    input_ids = input_ids.to(device)
    pos_ids = pos_ids.to(device)
    morph_ids = morph_ids.to(device)

    # logits = model(input_ids=input_ids, pos_ids=pos_ids, morph_ids=morph_ids).logits
    generated_ids = model.generate(input_ids=input_ids, pos_ids=pos_ids, morph_ids=morph_ids, num_beams=5, do_sample=False, no_repeat_ngram_size=3, max_length=256)
    # pred_input_ids = model.generate(input_ids=input_ids, pos_ids=pos_ids, morph_ids=morph_ids, do_sample=False, num_beams=5
    predicted_translations.extend(tokenizer.batch_decode(generated_ids, skip_special_tokens=True))

score = bertscore.compute(predictions=predicted_translations, references=test_y, lang="zh")
print("Bertscore: ", score)

Testing: 100%|██████████| 128/128 [04:24<00:00,  2.07s/it]


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Bertscore:  {'precision': [0.8111127018928528, 0.7177199125289917, 0.873715877532959, 0.8186167478561401, 0.8546887636184692, 0.7156949043273926, 0.8486291170120239, 0.8143483400344849, 0.7648950815200806, 0.7647104263305664, 0.8182451128959656, 0.6903266906738281, 0.8199440240859985, 0.8212870359420776, 0.7407934665679932, 0.7296829223632812, 0.7929103374481201, 1.0000001192092896, 0.7480124235153198, 0.7753587365150452, 0.8475520014762878, 0.8669065237045288, 0.8799730539321899, 0.8365086913108826, 0.7833747863769531, 0.7685627937316895, 0.77524334192276, 0.7477788329124451, 0.8461217880249023, 0.738330602645874, 0.8679195046424866, 0.7203071117401123, 0.9741601943969727, 0.6966577768325806, 0.957449197769165, 0.8162574768066406, 0.7558690905570984, 0.6981189250946045, 0.8348637223243713, 0.8084001541137695, 0.6896141767501831, 0.7835471630096436, 0.7388551831245422, 0.784774124622345, 0.911861002445221, 0.7158157229423523, 0.694398820400238, 0.8341413736343384, 0.9267975091934204, 0

In [ ]:
# anchibert tokenizer + bart-base-chinese
# print(predicted_translations[1016])
# print(test_y[1016])
# print(test_x[1016])

In [ ]:
# jihuai tokenizer
# print(predicted_translations[820])
# print(test_y[820])
# print(test_x[820])

不 要 帝 不 要 过 过 过 ， 过 了 过 了 ， 过 马 而
云升雨降多反复，错综变幻何纷繁。天地运转造万物，漫无边际何浩瀚。
文帝不听。居数年，怀王骑，堕马而死，无後。


In [ ]:
# bart base chinese
# print(len(predicted_translations[11]))
# print(predicted_translations[9])
# print(test_y[9])
# print(test_x[9])

33
诚 审 思 治 人 之 术 ， 欢 乐 得 贤 之 福 ， 论 材
您果真能仔细地考虑统治民众的方法，为有得到贤能的人的福气而欢悦，论才能选拔士人，一定要试用到职位上，明确规定考核标准来衡量他们的能力，考查功绩实际来评定他们的德行，不要采用结伙营私的人虚伪的赞美之辞，不要听受积渐深的诬陷之言，那么拥有功绩修正职责的官吏没有优点被隐蔽、受到中伤的忧虑，结党营私邪恶诡诈的人不能得到官位，小人一天天减少，俊杰治能之士一天天多起来。
诚审思治人之术，欢乐得贤之福，论材选士，必试于职，明度量以程能，考功实以定德，无用比周之虚誉，毋听浸润之谮诉，则抱功修职之吏无蔽伤之忧，比周邪伪之徒不得即工，小人日销，俊艾日隆。


In [30]:
# erya tokenizer
print(len(predicted_translations[1016]))
print(predicted_translations[1016])
print(test_y[1016])
print(test_x[1016])

51
从 哀 帝 、 平 帝 时 起 ， 各 郡 国 处 处 都 有 豪 杰 ， 然 而 没 有 多 少 。
自哀帝、平帝年间，郡国处处都有豪杰之士，然而数量却无法统计。
自哀、平间，郡国处处有豪桀，然莫足数。


In [61]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Thesis/arChSynBart_POS_MORPH.pt")

In [31]:
new = {}
new['precision'] = sum(score['precision']) / len(score['precision'])
new['recall'] = sum(score['recall']) / len(score['recall'])
new['f1'] = sum(score['f1']) / len(score['f1'])
print(new)

{'precision': 0.8092715767893057, 'recall': 0.7826650028998228, 'f1': 0.7953710840208897}


In [ ]:
print(morph_lookup)

defaultdict(<function <lambda> at 0x79b0f42fec20>, {'NameType=Prs': 0, '': 1, 'Person=1|PronType=Prs': 2, 'AdvType=Tim': 3, 'Case=Tem': 4, 'VerbType=Cop': 5, 'Case=Loc': 6, 'NameType=Giv': 7, 'Polarity=Neg': 8, 'Case=Loc|NameType=Nat': 9, 'Mood=Pot': 10, 'Case=Loc|NameType=Geo': 11, 'NameType=Sur': 12, 'Degree=Pos': 13, 'Mood=Des': 14, 'PronType=Prs': 15, 'Person=3|PronType=Prs': 16, 'Degree=Equ': 17, 'PronType=Int': 18, 'AdvType=Tim|Tense=Fut': 19, 'PronType=Dem': 20, 'Mood=Nec': 21, 'PronType=Prs|Reflex=Yes': 22, 'AdvType=Tim|Aspect=Perf': 23, 'AdvType=Deg|Degree=Sup': 24, 'AdvType=Tim|Tense=Past': 25, 'NounType=Clf': 26, 'Person=2|PronType=Prs': 27, 'AdvType=Cau': 28, 'Voice=Pass': 29, 'NumType=Ord': 30, 'AdvType=Tim|Tense=Pres': 31, 'AdvType=Deg|Degree=Pos': 32, 'AdvType=Deg|Degree=Cmp': 33})


In [ ]:
# sent = tokenizer("不父乙咎妇好")['input_ids']
# po = torch.tensor([101,4,0,0,2,0,0,102])
# mo = torch.tensor([101,8,1,1,0,1,1,102])
# p = model.generate(input_ids=torch.tensor([sent]).to(device), pos_ids=po.to(device), morph_ids=mo.to(device))
# print(tokenizer.batch_decode(p, skip_special_tokens=True))

['乙 不 父 乙 咎 妇 好 。']


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [38]:
print(pos_lookup)

defaultdict(<function <lambda> at 0x7b3729ffacb0>, {'PROPN': 0, 'NOUN': 1, 'VERB': 2, 'CCONJ': 3, 'PRON': 4, 'ADV': 5, 'SCONJ': 6, 'PART': 7, 'PUNCT': 8, 'AUX': 9, 'NUM': 10, 'SYM': 11, 'ADP': 12, 'INTJ': 13})


In [ ]:
!pip install suparkanbun
import suparkanbun
# doc=nlp("不入虎穴不得虎子")
import deplacy
# deplacy.render(doc)
# deplacy.serve(doc,port=None)
nlp=suparkanbun.load(BERT="roberta-classical-chinese-base-char",Danku=True)

In [64]:
sent = "不父乙咎妇好"
doc=nlp(sent)
p_list = [101]
m_list = [101]
for tok in doc:
    p = pos_lookup[str(tok.pos_)]
    m = morph_lookup[str(tok.morph)]
    p_list.append(p)
    m_list.append(m)
p_list.append(102)
m_list.append(102)
print(p_list)
print(m_list)

INFO:supar:Loading the data
INFO:supar:
Dataset(n_sentences=2, n_batches=2, n_buckets=2)
INFO:supar:Making predictions on the dataset
  0%|                  | 0/2 00:00<?, ?it/s/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:28: UserWarning: apply_permutation is deprecated, please use tensor.index_select(dim, permutation) instead
  warnings.warn("apply_permutation is deprecated, please use tensor.index_select(dim, permutation) instead")
INFO:supar:0:00:00.057017s elapsed, 35.08 Sents/s


[101, 5, 0, 0, 2, 1, 2, 102]
[101, 8, 7, 7, 1, 1, 1, 102]


In [65]:

sent = tokenizer(sent)['input_ids']
po = torch.tensor(p_list)
mo = torch.tensor(m_list)
p = model.generate(input_ids=torch.tensor([sent]).to(device), pos_ids=po.to(device), morph_ids=mo.to(device), max_length=256,num_beams=5, do_sample=True)
print(tokenizer.batch_decode(p, skip_special_tokens=True))

['不 父 乙 癸 妇 好']


In [ ]:
# import json
# with open('arChSynBart.json', 'w', encoding='utf-8') as f:
#     json.dump(new, f, ensure_ascii=False, indent=4)
#     f.write('\n')
#     f.write('epoch=5, lr=1e-5, num_beams=5\n')
#     f.write(model.config.to_json_string())

In [62]:
import json
# with open('arChSynBart.json', 'a', encoding='utf-8') as f:
#     f.write('above are the results of anchibert tokenizer + bert-base-chinese tokenizer\n')
#     f.write('\n the following are of jihuai tokenizer')
#     json.dump(new, f, ensure_ascii=False, indent=4)
#     f.write('\n')
#     f.write('epoch=5, lr=1e-5, num_beams=5\n')
#     f.write(model.config.to_json_string())

In [63]:
with open('arChSynBart-erya-tokenizer.json', 'a', encoding='utf-8') as f:
    # f.write('\n the following are of erya tokenizer')
    # json.dump(new, f, ensure_ascii=False, indent=4)
    # f.write('\n')
    # f.write('epoch=10, batch_size=16, lr=1e-5, num_beams=5\n')
    # f.write(model.config.to_json_string())
    f.write('\n即此言爱人者必见爱也: 就 是 这 样 说 爱 人 的 人 一 定 会 被 人 爱 戴。')